# 딥러닝 실습 환경 소개

In [1]:
import tensorflow as tf

# 하이퍼파라미터 설정

In [2]:
# Specify hyperparameters
EPOCHS = 5

# 네트워크 구조 정의

In [3]:
# Define network architecture
class MyModel(tf.keras.Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.flatten = tf.keras.layers.Flatten(input_shape=(28, 28))
        self.d1 = tf.keras.layers.Dense(128, activation='sigmoid')
        self.d2 = tf.keras.layers.Dense(10, activation='softmax')

    def call(self, x):
        x = self.flatten(x)
        x = self.d1(x)
        return self.d2(x)

# 학습 루프 정의

In [4]:
# Implement training loop
@tf.function
def train_step(model, images, labels, loss_object, optimizer, train_loss, train_accuracy):
    with tf.GradientTape() as tape:
        predictions = model(images)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)

    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    train_loss(loss)
    train_accuracy(labels, predictions)

# 테스트 루프 정의

In [5]:
# Implement algorithm test
@tf.function
def test_step(model, images, labels, loss_object, test_loss, test_accuracy):
    predictions = model(images)

    t_loss = loss_object(labels, predictions)
    test_loss(t_loss)
    test_accuracy(labels, predictions)

# 학습 데이터셋 가져오고 정리

In [6]:
# Import and organize dataset
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

# 네트워크 생성

In [7]:
# Create model
model = MyModel()

# 손실함수, 최적화 알고리즘 정의

In [8]:
# Define loss and optimizer
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

# 알고리즘 평가지표 설정

In [9]:
# Define performance metrics
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

# 알고리즘을 학습하고 테스트 루프

In [10]:
# Do training loop and test
for epoch in range(EPOCHS):
    for images, labels in train_ds:
        train_step(model, images, labels, loss_object, optimizer, train_loss, train_accuracy)

    for test_images, test_labels in test_ds:
        test_step(model, test_images, test_labels, loss_object, test_loss, test_accuracy)

    template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
    print(template.format(epoch + 1,
                          train_loss.result(),
                          train_accuracy.result() * 100,
                          test_loss.result(),
                          test_accuracy.result() * 100))
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()

Epoch 1, Loss: 0.3998907804489136, Accuracy: 89.60499572753906, Test Loss: 0.22880102694034576, Test Accuracy: 93.44000244140625
Epoch 2, Loss: 0.19798074662685394, Accuracy: 94.33833312988281, Test Loss: 0.16962897777557373, Test Accuracy: 94.93000030517578
Epoch 3, Loss: 0.1439712792634964, Accuracy: 95.87666320800781, Test Loss: 0.132237046957016, Test Accuracy: 95.9800033569336
Epoch 4, Loss: 0.11136715859174728, Accuracy: 96.7933349609375, Test Loss: 0.11351685971021652, Test Accuracy: 96.56999969482422
Epoch 5, Loss: 0.08955639600753784, Accuracy: 97.43833923339844, Test Loss: 0.0984702929854393, Test Accuracy: 97.05999755859375
